In [9]:
from networkx import adjacency_matrix, all_neighbors
from scipy.sparse import dia_matrix
from numpy import copy, full, zeros, array
from numpy.linalg import norm
from random import choice, random, sample
from itertools import count

from numpy import set_printoptions
from sys import maxsize

In [2]:
def simulateInfection(G, src, model='SI', lamda=0.3, runtime=1000, threshold=0.3):
    N = G.number_of_nodes()
    nodes = list(G.nodes())
    infected_nodes = {src}

    for i in range(runtime):
        temp_infected = infected_nodes.copy()

        for node in infected_nodes:
            for neighbour in all_neighbors(G, node):
                if random() < lamda:
                    temp_infected.add(neighbour)

        infected_nodes = temp_infected

        if len(infected_nodes) > threshold * N:
            break

    y = full(N, -1)
    for infNode in infected_nodes:
        y[nodes.index(infNode)] = 1

    return y

In [4]:
def labelRankingScore(G, y, alpha=0.5):
    N = G.number_of_nodes()

    W = adjacency_matrix(G)
    diag_elem = W.sum(axis=1).A1 ** (-0.5)

    inv_sqrt_D = dia_matrix((diag_elem, [0]), shape=W.get_shape())
    S = inv_sqrt_D @ W @ inv_sqrt_D

    f = copy(y)
    
    runtime = 0
    for t in count(0):
        f_ = alpha * S @ f + (1 - alpha) * y

        if norm(f - f_) < 0.001 * N: # Convergence Criteria
            runtime = t
            break

        f = f_

    return f, runtime

In [6]:
# known_dicts = [ {infected: [nodes], safe: [nodes]} ]

def simulatePartialInfection(
    G, src, model='SI', lamda=0.3,
    runtime_cap=1000, threshold=0.3,
    sampling=0.7, n_snapshot = 1   
):
    N = G.number_of_nodes()
    nodes = list(G.nodes())
    infected_nodes = {src}
    
    runtime = 1
    for t in range(runtime_cap):
        temp_infected = infected_nodes.copy()

        for node in infected_nodes:
            for neighbour in all_neighbors(G, node):
                if random() < lamda:
                    temp_infected.add(neighbour)

        infected_nodes = temp_infected

        if len(infected_nodes) > threshold * N:
            runtime = t
            break

    snapshots = [sample(nodes, int(sampling * N)) for _ in range(n_snapshot)]
    known_dicts = [{'infected': [],'safe': []} for _ in range(n_snapshot)]

    for i, snapshot in enumerate(snapshots):
        for node in snapshot:
            known_dicts[i]['infected' if node in infected_nodes else 'safe'].append(node)
                        
    return known_dicts, t

In [23]:
def resetF(F, known_dict):
    for node in known_dict['safe']:
        F[node][0] = 1
        F[node][1] = 0

    for node in known_dict['infected']:
        F[node][0] = 0
        F[node][1] = 1

def GFHF(G, known_dict):
    N = G.number_of_nodes()
    W = adjacency_matrix(G)

    diag_elem = 1 / W.sum(axis=1).A1
    inv_D = dia_matrix((diag_elem, [0]), shape=W.get_shape())

    P = inv_D @ W
    F = zeros((N, 2))

    resetF(F, known_dict)
    prev_diff = 0

    runtime = 0
    for t in count(0):
        F_ = P @ F
        curr_diff = sum(sum(abs(F - F_))) # Convergence Criteria

        resetF(F, known_dict)

        if curr_diff < 0.0001 * N:
            runtime = t
            break

        F = F_

    O = array([1 if f[1] > f[0] else -1 for f in F])
    return O, runtime

def LGC(G, known_dict, alpha):
    N = G.number_of_nodes()
    W = adjacency_matrix(G)

    diag_elem = W.sum(axis=1).A1 ** (-0.5)
    inv_sqrt_D = dia_matrix((diag_elem, [0]), shape=W.get_shape())

    S = inv_sqrt_D @ W @ inv_sqrt_D
    F = zeros((N, 2))

    resetF(F, known_dict)
    Y = copy(F)

    runtime = 0
    for t in count(0):
        F_ = alpha * S @ F + (1 - alpha) * Y

        if sum(sum(abs(F - F_))) < 0.0001 * N: # Convergence Criterion
            runtime = t
            break

        F = F_

    O = array([1 if f[1] > f[0] else -1 for f in F])
    return O, runtime

In [25]:
def detected_src(G, f):
    src_idx = max(enumerate(f), key=lambda x: x[1])
    src = list(G.nodes())[src_idx[0]]
    return src

In [29]:
import networkx as nx
from networkx import shortest_path_length as path
from pprint import pprint

In [48]:
G = nx.random_graphs.barabasi_albert_graph(600, 8)
src = choice(list(G.nodes()))
N = G.number_of_nodes()

known_dicts, sim_time = simulatePartialInfection(G, src, threshold=0.4, sampling=0.5, n_snapshot = 8)

# Return is [ ([], time) ]
GFHF_pred_time = [GFHF(G, labelled) for labelled in known_dicts]
LGC_pred_time = [LGC(G, labelled, alpha=0.5) for labelled in known_dicts]

GFHF_label_time = [labelRankingScore(G, y) for (y, t) in GFHF_pred_time]
LGC_label_time = [labelRankingScore(G, y) for (y, t) in LGC_pred_time]

# Return is [ dist actual_src -> predicted_src ]
GFHF_src_hop = [path(G, src, detected_src(G, f)) for (f, t) in GFHF_label_time]
LGC_src_hop = [path(G, src, detected_src(G, f)) for (f, t) in LGC_label_time]

# GFHF_error = 

pprint(f'GFHF Err: {GFHF_src_hop}')
pprint(f'LGC Err: {LGC_src_hop}')

'GFHF Err: [2, 2, 3, 2, 2, 2, 2, 2]'
'LGC Err: [3, 1, 2, 2, 3, 3, 2, 3]'
